# Step 1

## Get links to scripts

In [252]:
from bs4 import BeautifulSoup
import urllib.request

def fetch(url):
    request = urllib.request.Request(url)
    try:
        read_request = urllib.request.urlopen(request).read()#.decode('utf-8')
        return read_request
    except urllib.error.HTTPError:
        return False
    
def movie_title_from_hyperlink(hyperlink):
    return hyperlink["href"][len(script_link_first_part):-len(script_link_last_part)].replace(":", "")

script_folder_url = 'http://www.imsdb.com/scripts/'
genre_folder_url = 'http://www.imsdb.com/genre/'

script_link_first_part = "/Movie Scripts/"
script_link_last_part = " Script.html"

genres = [
    "Action",
    "Adventure",
    "Animation",
    "Comedy",
    "Drama",
    "Family",
    "Fantasy",
    "Horror",
    "Musical",
    "Mystery"
    "Romance",
    'Sci-Fi',
    'Thriller',
    "Western"
]


# A dictionary with movie title as key. Each movie in the dictionary
# links to a list with all of its genres
movie_dict = dict([])

for i, genre in enumerate(genres):
    
    # Crawl IMSDb for genre
    genre_url = genre_folder_url + genre
    request_text = fetch(genre_url)
    soup = BeautifulSoup(request_text, "html5lib")
    genre_hyperlinks = soup.find_all("a", href=True)
    
    # Get all movies from genre
    for hyperlink in genre_hyperlinks:
        if script_link_first_part in hyperlink["href"]:
            title = movie_title_from_hyperlink(hyperlink)
            
            if title not in movie_dict:
                movie_dict[title] = [0]*len(genres)
            movie_dict[title][i] = 1
    

print("Number of links: %d" % ( len(movie_dict) ))
print(movie_dict)

Number of links: 1167
{'Star Wars The Empire Strikes Back': [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], "She's Out of My League": [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Program, The': [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], "Avventura, L' (The Adventure)": [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0], 'Gandhi': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'Miracle Worker, The': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'Being There': [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'Gangs of New York': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'Rescuers Down Under, The': [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 'Speed Racer': [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 'Capote': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'Oblivion': [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'Producer, The': [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'Harry Potter and the Prisoner of Azkaban': [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], "Man Who Wasn't There, The": [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'Into the Wild': [0, 1, 0,

## Iterate over all links and get scripts

In [268]:
def get_movie_script(movie_title):
    
    script_url = script_folder_url + movie_title.strip(":").replace(" ","-") + ".html"
    request_text = fetch(script_url)
    
    if not request_text: # Check if we got error when attempting to fetch script url
        return ""
    
    soup = BeautifulSoup(request_text, "html5lib")
    
    script = soup.findAll("pre") #
    script_text = ""
    
    if script:
        script_text = script[-1].get_text() # Get all text within most inner "<pre>...</pre>"
    
    return script_text

# Same structure as movie_dict, but with s
scripts = []
labels = []

for i, movie_title in enumerate(movie_dict.keys()):
    
    #if i < 200:
    script = get_movie_script(movie_title)
    if script != "": # Check if the script was found
        scripts.append(script)
        labels.append(movie_dict[movie_title])

    #lines = script.get_text()#.split("<title>",1)[-1]#.splitlines()
    #print(lines)
    #descriptions.append(description)
    
    if i % 5 == 0: # Just to see progress
        print(i)

scripts_and_labels = {"scripts":scripts, "labels":labels}

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
765
770
775
780
785
790
795
800
805
810
815
820
825
830
835
840
845
850
855
860
865
870
875
880
885
890
895
900
905
910
915
920
925
930
935
940
945
950
955
960
965
970
975
980
985
990
995
1000
1005
1010
1015
1020
1025
1030
1035
1040
1045
1050
1055
1060
1065
1070
1075
1080
1085
1090
1095
1100
1105
1110
1115
1120
1125
1130
1135
1140
1145
1150
1155
1160
1165


# Save/Load scripts to file to save time re-crawling

### Save scripts

In [269]:
import pickle
pickle.dump(scripts_and_labels , open( "pickled_scripts_and_labels.p", "wb" ))

### Load scripts

In [270]:
import pickle
scripts_and_labels = pickle.load(open( "pickled_scripts_and_labels.p", "rb" ))

# Step 2

In [271]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
stops = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rasmu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rasmu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Preprocess all scripts

In [272]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

def preprocess(doc, stops):
    doc = doc.split()
    # • Remove non alpha-numeric characters
    # • Tokenize
    # • Lowercase words
    # • Remove stopwords
    # • Stem
    doc = [lmtzr.lemmatize(re.sub(r'\W+', '', word).lower()) for word in doc if word not in stops]
    return ' '.join(doc)

preprocessed_scripts = []
for script in scripts_and_labels["scripts"]:
    preprocessed_scripts.append(preprocess(script, stops))

print(preprocessed_scripts[0])


the empire strike back written lawrence kasdan leigh brackett from story george lucas ext galaxy  planet hoth a star destroyer move space releasing imperial probe robot underside one probe zoom toward planet hoth land ice covered surface an explosion mark point impact ext hoth  meteorite crater  snow plain  day a weird mechanical sound rise whining wind a strange probe robot several extended sensor emerges smokeshrouded crater the ominous mechanical probe float across snow plain disappears distance ext plain of hoth  day a small figure gallop across windswept ice slope the bundled rider mounted large gray snow lizard tauntaun curving plume snow rise beneath speeding paw twolegged beast the rider gallop slope rein lizard stop pulling protective goggles luke skywalker notice something sky he take pair electrobinoculars utility belt see smoke rising probe robot crashed the wind whip luke furlined cap activates comlink transmitter his tauntaun shift moan nervously beneath him luke into com

## Compute tfidf

In [273]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(stop_words = "english", min_df = 2)

X_train_counts = count_vect.fit_transform(preprocessed_scripts)
#print(X_train_counts.shape)
#print(X_train_counts)
#print(count_vect.vocabulary_)


transvector = TfidfVectorizer(stop_words = "english", min_df = 2)

scripts_tfidf = transvector.fit_transform(preprocessed_scripts)
#print(scripts_tfidf)
#print(scripts_tfidf)

# Split into training and test

In [274]:
import numpy as np
cut_point = int(scripts_tfidf.shape[0]/2) # split 50-50 train test

x_train = scripts_tfidf[:cut_point]
y_train = np.array(scripts_and_labels["labels"][:cut_point])

x_test = scripts_tfidf[cut_point:]
y_test =  np.array(scripts_and_labels["labels"][cut_point:])

print("Successfully split data into training and test")

Successfully split data into training and test


# Training a classifier

In [276]:
from skmultilearn.problem_transform import LabelPowerset, BinaryRelevance, ClassifierChain
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

classifier = LabelPowerset(MultinomialNB(), require_dense = False)

classifier.fit(x_train, y_train)

predictions = classifier.predict(x_test)

accuracy = accuracy_score(y_test,predictions)
print("accuracy:", accuracy)

for i, y_t in enumerate(y_test):
    print("y_t:", y_t, "predict:", str([predictions[i,x] for x in range(len(genres))]))


acc 0.194642857143
y_t: [1 0 0 0 0 0 0 0 0 0 0 1 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 1 0 1 0 0 0 0 0 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 1 0 0 0 0 0 0 1 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 1 0 0 0 0 0 0 0 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 1 0 0 0 0 0 0 0 0 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 1 0 0 0 0 1 0 0 0 0 0 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 0 0 1 1 0 0 0 1 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 1 0 0 0 0 0 0 0 0 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 1 0 0 0 0 0 0 0 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [1 1 0 0 1 0 0 0 0 0 0 1 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 1 0 0 0 0 0 0 1 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [1 1 0 0 0 0 1 0 0 0 0 0 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 

y_t: [0 0 0 0 0 0 1 1 0 0 0 1 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 0 0 0 1 0 0 0 1 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [1 1 0 0 0 0 0 0 0 0 0 0 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 0 0 0 0 0 0 0 1 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [1 0 0 1 1 0 0 0 0 0 0 0 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 1 0 0 1 0 0 0 1 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 1 0 0 0 0 0 0 0 0 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 1 0 0 0 0 0 0 1 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 1 0 0 0 0 0 0 0 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 1 0 0 0 0 0 1 1 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 1 1 0 0 0 0 0 0 0 0] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 1 0 0 0 0 0 0 0 1] predict: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
y_t: [0 0 0 0 1 